In [4]:
import pandas as pd, numpy as np, geopandas as gp

## Looking at DOR difference between large and small dams

In [5]:
gdrive = "/Volumes/GoogleDrive/My Drive/Condon_Research_Group/Research_Projects/Rachel/Research/Data/bifurcation_data_repo/"

In [2]:
large_dams = pd.read_csv(gdrive+"grand_threshold/all_basins_segGeo.csv")
all_dams = pd.read_csv(gdrive+"HPC_runs_fixed/all_basins_unfiltered/all_basins_segGeo.csv")

/Users/rachelspinti/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
large_dams = large_dams[["Hydroseq", "DOR", "geometry"]].sort_values("Hydroseq")
all_dams = all_dams[["Hydroseq", "DOR", "geometry"]].sort_values("Hydroseq")

In [ ]:
all_dams["DOR_diff"] = large_dams["DOR"]/all_dams["DOR"]

In [ ]:
len(all_dams.DOR_diff.unique())
# all_dams = all_dams.fillna(0)
# all_dams.to_csv(gdrive+"grand_threshold/dor_diff.csv")

In [ ]:
# print(large_dams.head(10))
# print(all_dams.head(10))
large_dams.columns()

## Looking at % storage by purpose

In [7]:
huc8_flood = gp.read_file(gdrive+"HPC_runs_fixed/flood_control/huc8_indices.shp")
# huc8_flood = gp.read_file(gdrive+"HPC_runs_fixed/water_supply/huc8_indices.shp")
huc8_all = gp.read_file(gdrive+"HPC_runs_fixed/all_basins_unfiltered/huc8_indices.shp")
print("flood = ", len(huc8_flood), "all = ", len(huc8_all))

flood =  2192 all =  2192


In [ ]:
huc8_flood = huc8_flood[["HUC8_no", "geometry", "Norm_sto_2"]]
huc8_all["Purposes"] = 'Nan'
huc8_all["stor_diff"] = 0
# huc8_all

In [3]:
for i in huc8_all.index:
    if huc8_flood.HUC8_no[i] == huc8_all.HUC8_no[i]:
        huc8_all["stor_diff"][i] = huc8_flood.Norm_sto_2[i]/huc8_all.Norm_sto_2[i]
#         huc8_all["Purposes"][i] = "Flood control"

NameError: name 'huc8_all' is not defined

In [15]:
type(huc8_all)

geopandas.geodataframe.GeoDataFrame

In [ ]:
huc8_all.to_file(gdrive+"HPC_runs_fixed/water_supply/huc8_supply_test.shp")

## Grand flag

In [2]:
# folder = 'purpose_analysis/flood_control/'
gdrive = "/Volumes/GoogleDrive/My Drive/Condon_Research_Group/Research_Projects/Rachel/Research/Data/bifurcation_data_repo/"
    
## NABD
nabd_dams = gp.read_file(gdrive+"nabd/nabd_fish_barriers_2012.shp", 
                            usecols=['COMID', 'NIDID', 'Norm_stor', 'Max_stor', 
                                    'Year_compl', 'Purposes', 'geometry'])  #read in NABD from Drive
nabd_dams = nabd_dams.drop_duplicates(subset='NIDID', keep="first")  #drop everything after first duplicate
print('Length of old NABD', len(nabd_dams))

#Add missing dams to NABD
dams_to_add = gp.read_file(gdrive+'other_dam_datasets/dams_to_add_centroid.shp')
dams_to_add = dams_to_add[['NIDID','Norm_stor', 'Max_stor', 'Year_compl', 
                                'Purposes', 'join_COMID', 'geometry']].rename(columns={'join_COMID':'COMID'}) #filter added dams to match NABD
nabd_dams = dams_to_add.append(nabd_dams)
print('Length of new NABD', len(nabd_dams))

#Update wrong NIDIDs for large dams
wrong_id = pd.read_csv(gdrive+'other_dam_datasets/large_dams_wrongID.csv', index_col = 0)
wrong_id = wrong_id[wrong_id['NABD_NIDID'].notna()]
wrong_id = wrong_id['NIDID']
nabd_dams.update(wrong_id)


#Add columns to NABD df for data processing/analysis
nabd_dams['COMID'] = pd.to_numeric(nabd_dams['COMID'])
nabd_dams["DamID"] = range(len(nabd_dams.COMID))  #add DamID 
nabd_dams = pd.DataFrame(nabd_dams)
nabd_dams['grand_flag'] = np.zeros(len(nabd_dams))  #add flag column

Length of old NABD 51795
Length of new NABD 51886


In [3]:
# nabd_dams.head(10)
# nabd_dams.Grand_flag.unique()

In [10]:
## GRanD
grand = pd.read_csv(gdrive+"other_dam_datasets/Reservoir_Attributes.csv", 
                            usecols=['GRAND_ID', 'NABD_ID'])  #read in NABD from Drive
#Filter out dams without NABD IDs
grand['NABD_ID'] = grand['NABD_ID'].fillna(0)
grand = grand[grand['NABD_ID']!=0]

#Merge NABD and GRanD
nabd = pd.merge(nabd_dams, grand, left_on = 'NIDID', right_on = 'NABD_ID', how = 'left')
nabd['GRAND_ID'] = nabd['GRAND_ID'].fillna(0)
# if nabd.loc[nabd.GRAND_ID != 0]:
#     nabd["Grand_flag"] = 1 #if a GRanD ID exists, make flag =1 
nabd.loc[nabd.GRAND_ID !=0, "grand_flag"]=1
nabd = nabd.drop(columns=['GRAND_ID', 'NABD_ID'])

In [11]:
print(nabd.grand_flag.unique())

[0. 1.]


## Working with pivot table and dam purpose

In [1]:
import pandas as pd, geopandas as gp, numpy as np

In [12]:
folder = 'HPC_runs_fixed/all_basins_unfiltered/'
gdrive = "/Volumes/GoogleDrive/My Drive/Condon_Research_Group/Research_Projects/Rachel/Research/Data/bifurcation_data_repo/" 
basin = "Red"
segments = pd.read_csv(gdrive + folder + basin + ".csv", index_col='Hydroseq',
                  usecols=['Hydroseq', 'UpHydroseq', 'DnHydroseq',
                            'LENGTHKM', 'StartFlag', 'DamCount',
                            'Coordinates', 'DamID',  'QC_MA', 'Norm_stor',
                            'HUC2', 'HUC4', 'HUC8', 'StreamOrde', 'Purposes', 'FTYPE'])

In [10]:
 # 6. Aggregate by HUC
#Aggregate segment values first
# HUC_vallist=['HUC2','HUC4','HUC8']
HUC_vallist=['HUC8']

for HUC_val in HUC_vallist:
    # print(HUC_val)
    # HUC_val = 'HUC8' # choices are HUC2, HUC4, HUC*

    # Summarize values in the segments table
    HUC_summary = segments.pivot_table(values=['Norm_stor', 'DamCount', 'LENGTHKM', 'Purposes'],
                                      index=HUC_val, aggfunc={'Norm_stor': (np.sum, np.max),
                                                                'DamCount': np.sum,
                                                                'LENGTHKM': np.sum,
                                                             'Purposes': lambda x:x.value_counts().index[1]})

    #Fixing pivot table columns
    HUC_summary.columns = ["_".join((i,j)) for i,j in HUC_summary.columns]
    HUC_summary.reset_index()

    # print('HUC summary test 1')
    # print(HUC_summary.columns)
    # Then grab variables from the fragments table
    HUC_summaryf = fragments.pivot_table(values=['LENGTHKM'],  index=HUC_val, 
                                         aggfunc={'LENGTHKM': (np.mean, len, np.max)})

    HUC_summaryf.columns = ["_".join((i,j)) for i,j in HUC_summaryf.columns]
    HUC_summaryf.reset_index()

    HUC_summary = pd.concat([HUC_summary, HUC_summaryf], axis=1)

    # print('HUC summary test 2')
    # print(HUC_summary.columns)

    # Identify the most downstream segment in each HUC based on the upstream segment length
    seg_group = segments.groupby(HUC_val)

    # Identify the segment_outlet
    seg_outlet = seg_group.LENGTHKM_up.idxmax() 
    HUC_summary['seg_outlet'] = seg_group.LENGTHKM_up.idxmax() #segment 'outlet'

    # print('HUC summary test 3')
    # print(HUC_summary.columns)

    #Use the segment outlet to look up other columns of interest
    column_list = ['Frag', 'LENGTHKM_up', 'DOR', 'dci', 'Norm_stor_up', 'QC_MA']
    outlet_vals = segments.loc[HUC_summary.seg_outlet, column_list]
    HUC_summary = HUC_summary.join(outlet_vals, on='seg_outlet', rsuffix='_outlet')
    
    # print('HUC summary test 4')
    # print(HUC_summary.columns)

    # HUC_summary = HUC_summary[column_list].add_suffix('_outlet')
    add_suffix = [(i, i+'_outlet') for i in column_list]
    HUC_summary.rename(columns = dict(add_suffix), inplace=True)
        
    # print('HUC summary test 5')
    # print(HUC_summary.columns)

    # write out to csv
    HUC_summary.to_csv(gdrive+folder+basin+HUC_val+'_indices.csv')
    print('Finished writing huc '+HUC_val+' indices to csv')

IndexError: index 1 is out of bounds for axis 0 with size 1

In [20]:
test = segments[['HUC8', 'Purposes', 'DamID']]
test
'Purposes': lambda x:x.value_counts().index[1]

,HUC8,Purposes,DamID
Hydroseq,,,
840002063,9020101.0,Flood control,19876.0
840000151,9020301.0,Water Supply,28287.0
840003868,9020102.0,Water Supply,19879.0
840028840,9020102.0,Water Supply,19880.0
840004526,9020103.0,Other,19881.0
...,...,...,...
590069290,9040002.0,NaN,0.0
590109874,9040002.0,NaN,0.0
590096771,9040002.0,NaN,0.0


In [17]:
segments.Purposes.unique()

array(['Flood control', 'Water Supply', 'Other', 'Hydroelectric', nan],
      dtype=object)

In [18]:
folder = 'HPC_runs_fixed/all_basins_unfiltered/'
gdrive = "/Volumes/GoogleDrive/My Drive/Condon_Research_Group/Research_Projects/Rachel/Research/Data/bifurcation_data_repo/" 
basin = "Colorado"
segments_col = pd.read_csv(gdrive + folder + basin + ".csv", index_col='Hydroseq',
                  usecols=['Hydroseq', 'UpHydroseq', 'DnHydroseq',
                            'LENGTHKM', 'StartFlag', 'DamCount',
                            'Coordinates', 'DamID',  'QC_MA', 'Norm_stor',
                            'HUC2', 'HUC4', 'HUC8', 'StreamOrde', 'Purposes', 'FTYPE'])

/Users/rachelspinti/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [19]:
segments_col.Purposes.unique()

array(['Water Supply', 'Hydroelectric', 'Flood control', 'Other', nan],
      dtype=object)